In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

In [17]:
# 🔹 경로 및 모델명
file_path = "cleaned_dataset_half.json"
model_name = "beomi/KoAlpaca-Polyglot-5.8B"

# ✅ 1. 데이터 로드
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

dataset = Dataset.from_list(data)

In [18]:
# ✅ 2. 토크나이저 설정
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token  # LLaMA 계열에서 필수

# ✅ 3. 토크나이징 함수 정의
def tokenize(example):
    prompt = example["input"].strip()
    response = example["output"].strip()
    full_text = prompt + "\n" + response

    tokenized = tokenizer(
        full_text,
        truncation=True,
        max_length=512,
        padding="max_length"
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(tokenize, remove_columns=["input", "output"])

Map: 100%|██████████| 6353/6353 [00:01<00:00, 3989.13 examples/s]


In [19]:
# ✅ 4. 양자화 설정 + 모델 로드
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=False  # CPU offload 금지
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)


ImportError: Using `bitsandbytes` 8-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

In [9]:
# ✅ 5. LoRA 설정 및 적용
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["attention.query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)


model = get_peft_model(model, lora_config)


NameError: name 'model' is not defined

In [2]:
def generate(prompt, max_new_tokens):
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items() if k != "token_type_ids"}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.5,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
input_text = "오늘 굉장히 기분이 안좋아"
prompt = (
    "### 질문:\n"
    f"{input_text}"
    "### 답변: (공손한 말투로)\n"
)
generated_prompt = generate(prompt, 120)

# 콜론(:)이 나오는 위치를 모두 찾기
colon_indices = [i for i, c in enumerate(generated_prompt) if c == ")"]

# 두 번째 콜론 이후 문자열 자르기
if len(colon_indices) >= 1:
    result = generated_prompt[colon_indices[0] + 1:].strip()

    # 마침표가 다섯 번 나올 때까지 자르기
    dot_count = 0
    end_index = 0
    for i, c in enumerate(result):
        if c == '.':
            dot_count += 1
            if dot_count == 5:
                end_index = i + 1  # 마침표 포함
                break

    truncated_result = result[:end_index].strip()
    print(truncated_result)
else:
    print("콜론이 두 개 이상 없습니다.")


In [ ]:
from gtts import gTTS

# 언어 설정
language = 'ko'

# gTTS 객체 생성
speech = gTTS(text=result, lang=language, slow=False)

# 음성 파일로 저장
speech.save("tts_output_wav/output1.wav")
